# Fire and Patch Size Distribution Figures

In [ ]:
import os
import geopandas as gpd
import pandas as pd
import numpy as np

In [ ]:
TARGET_CRS = "EPSG:5070"

SQM_PER_HA = 10000

## Input Data
Load all fire perimeters and clip to the Westside

In [ ]:
data_dir = os.path.join("..", "data")

In [ ]:
westside = gpd.read_file(os.path.join(data_dir, "study_area.gpkg"))

In [ ]:
# Available at https://www.mtbs.gov/direct-download
mtbs_path = os.path.join(data_dir, "mtbs", "mtbs_perims_DD.shp")
mtbs = gpd.read_file(mtbs_path).to_crs(TARGET_CRS)
mtbs_westside = gpd.clip(mtbs, westside, keep_geom_type=True)

In [ ]:
nifc_2020_path = os.path.join(data_dir, "NIFC", "Public_NIFS_Perimeters_westside_ecoregions_complete.shp")
nifc_2020 = gpd.read_file(nifc_2020_path).to_crs(TARGET_CRS)
nifc_2020_westside = gpd.clip(nifc_2020, westside, keep_geom_type=True)

## Processing
Collapse the fire geodataframes into tables of area and year range, and combine

In [ ]:
fires_1985_2019 = pd.DataFrame({"area": mtbs_westside.area, "yr_range": "1985-2019"})
fires_1985_2019["ha"] = fires_1985_2019.area.divide(SQM_PER_HA)
fires_2020 = pd.DataFrame({"area": nifc_2020_westside.area, "yr_range": "2020"})
fires_2020["ha"] = fires_2020.area.divide(SQM_PER_HA)

## Fire Size Figure

In [ ]:
import plotly.express as px

In [ ]:
bins = [-np.inf, 100, 500, 1_000, 5_000, 10_000, 25_000, 50_000, np.inf]

# Manual labels corresponding to the bins for labeling x-axis ticks
ticktext = ["<100", "500", "1,000", "5,000", "10,000", "25,000", "50,000", ">50,000"]


# https://towardsdatascience.com/histograms-with-plotly-express-complete-guide-d483656c5ad7#c08f

fires_2020["count"] = pd.cut(fires_2020.ha, bins, include_lowest=True)
# Count the number of fires in each bin, then divide by the number of fires to get % of fires
agg2020 = fires_2020["count"].value_counts()
# # Convert to dataframe
agg2020 = agg2020.sort_index().to_frame().reset_index()
agg2020["bin_percent"] = agg2020["count"].divide(len(fires_2020))
agg2020["yr_range"] = "2020"

fires_1985_2019["count"] = pd.cut(fires_1985_2019.ha, bins, include_lowest=True)
# Count the number of fires in each bin, then divide by the number of fires to get % of fires
agg1985 = fires_1985_2019["count"].value_counts()
# Convert to dataframe
agg1985 = agg1985.sort_index().to_frame().reset_index()
agg1985["bin_percent"] = agg1985["count"].divide(len(fires_1985_2019))
agg1985["yr_range"] = "1985-2019"

# Combine the two year ranges
counts = pd.concat([agg1985, agg2020])
# rename index (containing the bin range e.g. "(5000000.0, 10000000.0]" to bins)
counts.rename(columns={"index":"size_bin"}, inplace=True)
# # Plotly cannot work with categories index, so we need to turn it into string
counts["size_bin"] = counts["size_bin"].astype("str")

In [ ]:
fire_fig = px.bar(
    counts, x="size_bin", y="bin_percent", color="yr_range",
    labels={"bin_percent":"Portion of fires", "size_bin": "Fire size class (ha)"},
    barmode="group", text="count",
    color_discrete_sequence=px.colors.qualitative.Set1[0:2][::-1]
)

fire_fig.update_layout(yaxis=dict(title_standoff=0), xaxis=dict(title_standoff=0))

fire_fig.update_layout(yaxis=dict(tickformat=",.0%"))

fire_fig.update_layout(
    template="ggplot2",
    plot_bgcolor='rgb(255, 255, 255)',
    legend=dict(
        title="",
        x=0.65, y=0.98,
        bordercolor="rgb(0,0,0)",
        borderwidth=0.5,
    ),
    bargap=0.2,
    bargroupgap=0.0
)

fire_fig.update_layout(showlegend=True)

# Black border around bars
fire_fig.update_traces(marker=dict(line=dict(color="black")), textposition="outside")

# Add solid plot border and grey gridlines
fire_fig.update_yaxes(showline=True, linecolor="black", linewidth=1, mirror=True, gridcolor="rgba(0, 0, 0, 0.1)")
fire_fig.update_xaxes(showline=True, linecolor="black", linewidth=1, mirror=True, gridcolor="rgba(0, 0, 0, 0.1)")

# Manual x axis value labels
fire_fig.update_xaxes(tickmode="array", tickvals=counts.size_bin, ticktext=ticktext, tickangle=45)

fire_fig.update_layout(width=500, height=450)

fire_fig

## Patch Size Figure

In [ ]:
# Folder of CSVs, by year, of individual patches. Processed by an R script in that directory
data_dir = os.path.join(data_dir, "patch_size")

In [ ]:
df_list = [pd.read_csv(os.path.join(data_dir, file)) for file in os.listdir(data_dir) if file.endswith(".csv")]
patch_df = pd.concat(df_list)

patch_df["yr_range"] = patch_df.yr.apply(lambda x : "2020" if x == 2020 else "1985-2019")

patch_bins = [-np.inf, 1, 10, 100, 1000, 10000, np.inf]
patch_ticktext = ["<1", "10", "100", "1,000", "10,000", ">10,000"]
patch_df["size_bin"] = pd.cut(patch_df["value"], patch_bins)

# Count the area in each patch size bin in each year group
patch_group_df = patch_df.groupby(["yr_range", "size_bin"])["value"].sum().reset_index()
# Calculate the percent of the year group's total fire area in each patch size bin
patch_group_df["bin_percent"] = patch_group_df["value"] / patch_group_df.groupby("yr_range")["value"].transform(sum)
# Plotly cannot work with categories index, so we need to turn it into string
patch_group_df["size_bin"] = patch_group_df["size_bin"].astype("str")

In [ ]:
patch_fig = px.bar(
    patch_group_df, x="size_bin", y="bin_percent", color="yr_range",
    labels={"bin_percent":"Portion of high severity fire extent", "size_bin": "High-severity patch size class (ha)"},
    barmode="group",
    color_discrete_sequence=px.colors.qualitative.Set1[0:2][::-1]
)

patch_fig.update_layout(yaxis=dict(title_standoff=0), xaxis=dict(title_standoff=0))

patch_fig.update_layout(yaxis=dict(tickformat=",.0%"))

patch_fig.update_layout(
    template="ggplot2",
    plot_bgcolor='rgb(255, 255, 255)',
    legend=dict(
        title="",
        x=0.015, y=0.98,
        bordercolor="rgb(0,0,0)",
        borderwidth=0.5,
    ),
    bargap=0.4,
    bargroupgap=0.0
)

# Black border around bars
patch_fig.update_traces(marker=dict(line=dict(color="black")))

# Hide the legend if the plot is going to be next to the fire area plot that has the same legend
patch_fig.update_layout(showlegend=False)

# Add solid plot border and grey gridlines
patch_fig.update_yaxes(showline=True, linecolor="black", linewidth=1, mirror=True, gridcolor="rgba(0, 0, 0, 0.1)")
patch_fig.update_xaxes(showline=True, linecolor="black", linewidth=1, mirror=True, gridcolor="rgba(0, 0, 0, 0.1)")

# Manual x axis value labels
patch_fig.update_xaxes(tickmode="array", tickvals=patch_group_df.size_bin, ticktext=patch_ticktext, tickangle=45)

patch_fig.update_layout(width=500, height=450)

patch_fig.show()